In [1]:
# Imports
import sys
import os
sys.path.append('../..')

# Data Manipulation
import pandas as pd
import openpyxl

In [2]:
# Global Variables
DATA_PATH = "../../data/"

## 1. Arquivos de Layout

In [3]:
# Carrega o arquivo de labels como um arquivo Excel
layout_2010 = pd.ExcelFile('2010_Layout_microdados_Amostra.xlsx')

In [4]:
# Funções Auxiliares

def remove_unnamed_columns(df) -> pd.DataFrame:
    """
    Remove colunas com nome Unnamed. Estas colunas vem de colunas que foram mescladas no .xlsx original.

    Args:
        df (pd.DataFrame): DataFrame com colunas Unnamed

    Returns:
        pd.DataFrame: DataFrame sem colunas Unnamed
    
    """
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


def get_values(split: list):
    values = {}
    for val in range(1, len(split)):
        aux = split[val].split('-')
        if len(aux) == 1:
            aux.append(aux[0])
            aux[0] = ' '    
        # aux[1] = aux[1].replace(' ', '')
        values[aux[0]] = aux[1]
    # print(values)
    return values


def split_col_names_values(col: pd.Series): 
    df = pd.DataFrame(columns = ['VAR', 'NOME', 'CHAVE', 'VALOR'])
    for index, value in col.items():
        df_aux = pd.DataFrame(columns = df.columns)
        split = value.split('\n')
        # df_aux['NOME'] = pd.concat([df_aux['NOME'], pd.Series(split[0])])
        if len(split) > 1:
            # series_values = pd.concat([series_values, pd.Series(get_values(split))])
            # df_line = pd.DataFrame(columns = ['NOME', 'CHAVES', 'VALORES'])
            # # df_line['NOME'] = pd.Series(split[0])
            # df_line = pd.Series(get_values)
            # dct = pd.Series(get_values)
            lst = [{'CHAVE': d_key, 'VALOR': d_value, 'VAR': index} for d_key, d_value in get_values(split).items()]
            df_aux = pd.DataFrame(lst)
            df_aux['NOME'] = split[0]
            
        else:
            df_aux['NOME'] = pd.Series(split[0])
            
        df = pd.concat([df, df_aux])
    return df


# def prepare_df(df: pd.DataFrame):
#     df_main = pd.DataFrame(columns = df.columns)
#     df = remove_unnamed_columns(df) 
#     for index, row in df.iterrows():
#         df_row = pd.DataFrame(columns = df.columns)
#         # for col in df.columns.drop('NOME'):
#         #     # print(entry)
#         #     df_row[col] = df[col]
#         df_row = pd.concat([df_row, split_col_names_values(row['NOME'])])
#         # print(df_entry)
#         df_aux = pd.concat([df_aux, df_row])
#     return df, df_aux

def prepare_df(df: pd.DataFrame)-> tuple:
    df = remove_unnamed_columns(df)
    df_vars = split_col_names_values(df.set_index('VAR')['NOME'])
    return df, df_vars

In [5]:
# Carrega as labels referentes aos microdados de domicílios
df_domi = pd.read_excel(layout_2010, sheet_name='DOMI')
df_domi.head()

,VAR,NOME,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,NaN,NaN,NaN,NaN,NaN,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,NaN,NaN,NaN,NaN,NaN,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,NaN,NaN,NaN,NaN,NaN,8,20,13,NaN,A
3,V0300,CONTROLE,NaN,NaN,NaN,NaN,NaN,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,NaN,NaN,NaN,NaN,NaN,29,44,3,13.0,N


In [6]:
df_domi, df_domi_vars = prepare_df(df_domi)
df_domi.head()

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N


In [7]:
df_domi_vars.head()

,VAR,NOME,CHAVE,VALOR
0,V0001,UNIDADE DA FEDERAÇÃO:,11,Rondônia
1,V0001,UNIDADE DA FEDERAÇÃO:,12,Acre
2,V0001,UNIDADE DA FEDERAÇÃO:,13,Amazonas
3,V0001,UNIDADE DA FEDERAÇÃO:,14,Roraima
4,V0001,UNIDADE DA FEDERAÇÃO:,15,Pará


In [8]:
# # Dataframe de pessoas
# df_pess = pd.read_excel(layout_2010, sheet_name='PESS')
# # Dataframe de emigração  
# df_emig = pd.read_excel(layout_2010, sheet_name='EMIG')
# # Dataframe de mortalidade
# df_mort = pd.read_excel(layout_2010, sheet_name='MORT')

In [9]:
MICRODATA_PATH = DATA_PATH + "microdados/"

dfs = {
    'amostra_domicilios_2010': df_domi,
    # 'amostra_pessoas_2010': df_pess,
    # 'amostra_emigracao_2010': df_emig,
    # 'amostra_mortalidade_2010': df_mort
}

dfs_vars = {
    'amostra_domicilios_2010': df_domi_vars,
    # 'amostra_pessoas_2010': df_pess_vars,
    # 'amostra_emigracao_2010': df_emig_vars,
    # 'amostra_mortalidade_2010': df_mort_vars
}

In [25]:
# Funções auxiliares

def split_by_index(
        input_string: str, 
        indexes: list[int]
    ) -> list[str]:
    """
    Splits a string into a list of strings using the indexes provided.
    
    Args:
        input_string (str): String to be split
        indexes (list[int]): List of indexes indicating where to split the string
        
    Returns:
        list[str]: List of splitted strings
    """
    return [input_string[i:j] for i, j in zip(indexes, indexes[1:] + [None])]


def extract_line_values(
        file: str, df: pd.DataFrame
    ) -> dict[int, list[str]]:
    """
    Extracts the values from the line of a microdata microdata file.

    Args:
        file (str): Name of the file to be processed
        df (pd.DataFrame): DataFrame containing the indexes of the columns

    Returns:
        dict[int, list[str]]: Dictionary containing the values of each line.
            The keys are the line numbers and the values are lists of strings containing the values not translated.
    """
    # open file
    with open(MICRODATA_PATH + file, 'r') as f:
        # create a counter for indexing the lines
        count = 0
        # create a dictionary to store the values
        values = {}
        # read all lines
        lines = f.readlines()
        # iterate over the lines
        for line in lines:
            # split the lines using the indexes from the dataframe
            line_values = split_by_index(line, df['POSIÇÃO INICIAL'].apply(lambda x: int(x) - 1).tolist())
            # save the values in the dictionary
            line_values[-1] = line_values[-1].replace('\n', '')
            values[count] = line_values 
            count += 1
    # close the file
        f.close()
    return values


def process_microdata_files(
        file_list: list[str], 
        df: dict[str, pd.DataFrame], 
        df_vars: dict[str, pd.DataFrame]
    ):
    """
    WORK IN PROGRESS. This function will process all the microdata files
    """
    for file in file_list:
        print(f"Iniciando a extração do arquivo [{file}]")
        values = extract_line_values(file, df)
        print(values)

    # for file in os.listdir(dir_path):
    #     if "amostra" not in file:
    #         continue
    #     # get the name and year of the research from the file name
    #     # the format is amostra_domicilios_YYYY_UF.txt
    #     research = '_'.join(file.split('_')[:3])
    #     try:
    #         # extract the lines from the file
    #         values = extract_line_values(file, dfs[research])
    #         # translate the values
    #         translate_microdata(values, dfs[research])
    #     except KeyError:
    #         # print(f"O arquivo {file} não foi processado.");
    #         continue


def translate_line_microdata(line_values: str, df: pd.DataFrame, df_vars: pd.DataFrame):
    df_empty = pd.DataFrame(columns = df_vars['NOME'].unique().tolist())
    line_dict = {}
    # for each line, translate the int values to they true values
    for i in range(len(line_values)):
        
        # get the value that will be translaed
        value = line_values[i]
        # get the line from the dataframe    
        df_line = df.iloc[i]
        # get the name of the column (the ith unique column name)
        df_line_col_name = df_vars['NOME'].unique().tolist()[i]

        line_dict['VAR'] = df_line['VAR']
        # verify if the value is "translatable"
        if df_line['TIPO'] == 'C' or df_line['VAR'] == 'V0001':
            # if it is, get the possible values
            possible_values = df_vars[df_vars['VAR'] == df_line['VAR']]
            # get the value from the possible values
            translated_value = possible_values[possible_values['CHAVE'] == value]
            try:
                line_dict[df_line_col_name] = str(translated_value['VALOR'][0])
            except KeyError:
                line_dict[df_line_col_name] = str(value)
        else:
            line_dict[df_line_col_name] = value
        df_line = pd.concat([df_empty, pd.DataFrame(line_dict, index=[0])])
    return df_line


In [11]:
df_domi.tail()

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
71,M0301,MARCA DE IMPUTAÇÃO NA V0301: \n1- Sim\n2- Não,168,168,1,NaN,C
72,M0401,MARCA DE IMPUTAÇÃO NA V0401: \n1- Sim\n2- Não,169,169,1,NaN,C
73,M0402,MARCA DE IMPUTAÇÃO NA V0402: \n1- Sim\n2- Não,170,170,1,NaN,C
74,M0701,MARCA DE IMPUTAÇÃO NA V0701: \n1- Sim\n2- Não,171,171,1,NaN,C
75,V1005,Situação do setor \n1 - Área urbanizada\n2 - Á...,172,172,1,NaN,C


In [26]:
translate_line_microdata(sc_line_dict[0], dfs['amostra_domicilios_2010'], dfs_vars['amostra_domicilios_2010'])

,UNIDADE DA FEDERAÇÃO:,CÓDIGO DO MUNICÍPIO,ÁREA DE PONDERAÇÃO,CONTROLE,PESO AMOSTRAL,REGIÃO GEOGRÁFICA:,CÓDIGO DA MESORREGIÃO:,CÓDIGO DA MICRORREGIÃO:,CÓDIGO DA REGIÃO METROPOLITANA:,SITUAÇÃO DO DOMICÍLIO:,...,MARCA DE IMPUTAÇÃO NA V0219:,MARCA DE IMPUTAÇÃO NA V0220:,MARCA DE IMPUTAÇÃO NA V0221:,MARCA DE IMPUTAÇÃO NA V0222:,MARCA DE IMPUTAÇÃO NA V0301:,MARCA DE IMPUTAÇÃO NA V0401:,MARCA DE IMPUTAÇÃO NA V0402:,MARCA DE IMPUTAÇÃO NA V0701:,Situação do setor,VAR
0,42,00051,4200051001001,00000745,0034868694751419,4,03,009,00,2,...,2,2,2,2,2,2,2,2,8,V1005


In [13]:
%%time
file = "amostra_domicilios_2010_SC.txt"
sc_line_dict = extract_line_values(file, dfs['amostra_domicilios_2010'])

CPU times: total: 42.9 s
Wall time: 48.1 s


In [ ]:
# sc_line_dict[0]

['42',
 '00051',
 '4200051001001',
 '00000745',
 '0034868694751419',
 '4',
 '03',
 '009',
 '00',
 '2',
 '01',
 '11',
 '5',
 '      ',
 '         ',
 '3',
 '06',
 '003',
 '01',
 '020',
 '1',
 ' ',
 '3',
 '01',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '1',
 '2',
 '02',
 '1',
 '2',
 '0003710',
 '0000727451',
 '00185500',
 '000363725',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '8']

In [ ]:
df_domi

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N
...,...,...,...,...,...,...,...
71,M0301,MARCA DE IMPUTAÇÃO NA V0301: \n1- Sim\n2- Não,168,168,1,NaN,C
72,M0401,MARCA DE IMPUTAÇÃO NA V0401: \n1- Sim\n2- Não,169,169,1,NaN,C
73,M0402,MARCA DE IMPUTAÇÃO NA V0402: \n1- Sim\n2- Não,170,170,1,NaN,C
74,M0701,MARCA DE IMPUTAÇÃO NA V0701: \n1- Sim\n2- Não,171,171,1,NaN,C


In [ ]:
# with open(MICRODATA_PATH + file, 'r') as f:
#     lines = f.readlines()
#     print(lines[0], lines[1], lines[2], lines[3], lines[4], sep='')